In [1]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [91.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,965 kB]
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
G

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [3]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("Spark").getOrCreate()
spark

In [4]:
from pyspark.sql import functions
print(dir(functions))


['Column', 'DataFrame', 'DataType', 'PandasUDFType', 'PythonEvalType', 'SparkContext', 'StringType', 'UserDefinedFunction', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', '_create_column_from_literal', '_create_lambda', '_create_udf', '_get_get_jvm_function', '_get_lambda_parameters', '_invoke_binary_math_function', '_invoke_function', '_invoke_function_over_column', '_invoke_higher_order_function', '_options_to_str', '_test', '_to_java_column', '_to_seq', '_unresolved_named_lambda_variable', 'abs', 'acos', 'acosh', 'add_months', 'aggregate', 'approxCountDistinct', 'approx_count_distinct', 'array', 'array_contains', 'array_distinct', 'array_except', 'array_intersect', 'array_join', 'array_max', 'array_min', 'array_position', 'array_remove', 'array_repeat', 'array_sort', 'array_union', 'arrays_overlap', 'arrays_zip', 'asc', 'asc_nulls_first', 'asc_nulls_last', 'ascii', 'asin', 'asinh', 'assert_true', 'atan', 'atan2', 'atanh', 'a

In [9]:
from pyspark.sql.functions import lit,lower, upper,concat,substring


df = spark.read.csv('/content/drive/MyDrive/data/insurance.csv', header=True, sep=",", inferSchema=True)
df.show(10)

+---+------+------+--------+------+---------+-----------+
|age|   sex|   bmi|children|smoker|   region|    charges|
+---+------+------+--------+------+---------+-----------+
| 19|female|  27.9|       0|   yes|southwest|  16884.924|
| 18|  male| 33.77|       1|    no|southeast|  1725.5523|
| 28|  male|  33.0|       3|    no|southeast|   4449.462|
| 33|  male|22.705|       0|    no|northwest|21984.47061|
| 32|  male| 28.88|       0|    no|northwest|  3866.8552|
| 31|female| 25.74|       0|    no|southeast|  3756.6216|
| 46|female| 33.44|       1|    no|southeast|  8240.5896|
| 37|female| 27.74|       3|    no|northwest|  7281.5056|
| 37|  male| 29.83|       2|    no|northeast|  6406.4107|
| 60|female| 25.84|       0|    no|northwest|28923.13692|
+---+------+------+--------+------+---------+-----------+
only showing top 10 rows



In [11]:
df.select(df.region, lower(df.region), upper(df.sex), substring(df.sex,1,1)).show()

+---------+-------------+----------+--------------------+
|   region|lower(region)|upper(sex)|substring(sex, 1, 1)|
+---------+-------------+----------+--------------------+
|southwest|    southwest|    FEMALE|                   f|
|southeast|    southeast|      MALE|                   m|
|southeast|    southeast|      MALE|                   m|
|northwest|    northwest|      MALE|                   m|
|northwest|    northwest|      MALE|                   m|
|southeast|    southeast|    FEMALE|                   f|
|southeast|    southeast|    FEMALE|                   f|
|northwest|    northwest|    FEMALE|                   f|
|northeast|    northeast|      MALE|                   m|
|northwest|    northwest|    FEMALE|                   f|
|northeast|    northeast|      MALE|                   m|
|southeast|    southeast|    FEMALE|                   f|
|southwest|    southwest|      MALE|                   m|
|southeast|    southeast|    FEMALE|                   f|
|southeast|   

In [13]:
df.select(df.age, df.sex, concat(df.age,lit(" "), df.sex)).alias("Age sex").show(2, truncate =False)

+---+------+-------------------+
|age|sex   |concat(age,  , sex)|
+---+------+-------------------+
|19 |female|19 female          |
|18 |male  |18 male            |
+---+------+-------------------+
only showing top 2 rows



In [15]:
from pyspark.sql.functions import min, max
df.dtypes

df.select(min(df.charges), max(df.charges)).show()
df.select(min(df.charges +lit(100)), max(df.charges +lit(500))).show()

+------------+------------+
|min(charges)|max(charges)|
+------------+------------+
|   1121.8739| 63770.42801|
+------------+------------+

+--------------------+--------------------+
|min((charges + 100))|max((charges + 500))|
+--------------------+--------------------+
|           1221.8739|         64270.42801|
+--------------------+--------------------+



In [16]:
def cal_age(date):
  age = date
  return age
  
from pyspark.sql.functions import to_date, to_timestamp
df1 =spark.createDataFrame([("2022-09-08 12:39:69",)], ["DATE"])
df1.show()
df1.printSchema()

df1.select(df1.DATE, to_date(df1.DATE), to_timestamp(df1.DATE)).show()

+-------------------+
|               DATE|
+-------------------+
|2022-09-08 12:39:69|
+-------------------+

root
 |-- DATE: string (nullable = true)

+-------------------+-------------+------------------+
|               DATE|to_date(DATE)|to_timestamp(DATE)|
+-------------------+-------------+------------------+
|2022-09-08 12:39:69|   2022-09-08|              null|
+-------------------+-------------+------------------+



In [20]:
_region =spark.createDataFrame([[1, "A"],[2, "f"],[3, "g"],[4, "g",]], ["id", "reg"])
_charges =spark.createDataFrame([[1, "12"],[2, "34"],[3, "54"],[4, "69",]], ["id", "char"])
_region.show()
_charges.show()
_region.join(_charges, _region.id == _charges.id, "inner").select(_region["id"],_region["reg"],_charges["char"]).show()


+---+---+
| id|reg|
+---+---+
|  1|  A|
|  2|  f|
|  3|  g|
|  4|  g|
+---+---+

+---+----+
| id|char|
+---+----+
|  1|  12|
|  2|  34|
|  3|  54|
|  4|  69|
+---+----+

+---+---+----+
| id|reg|char|
+---+---+----+
|  1|  A|  12|
|  3|  g|  54|
|  2|  f|  34|
|  4|  g|  69|
+---+---+----+



In [25]:
df = spark.read.csv('/content/drive/MyDrive/data/insurance.csv', header=True, sep=",", inferSchema=True)
df.show(2)

df.createOrReplaceTempView("tmp")

spark.sql("select * from tmp").show()
spark.sql("select age, bmi, region from tmp where bmi >25.0 and region == 'southwest'").show()

+---+------+-----+--------+------+---------+---------+
|age|   sex|  bmi|children|smoker|   region|  charges|
+---+------+-----+--------+------+---------+---------+
| 19|female| 27.9|       0|   yes|southwest|16884.924|
| 18|  male|33.77|       1|    no|southeast|1725.5523|
+---+------+-----+--------+------+---------+---------+
only showing top 2 rows

+---+------+------+--------+------+---------+-----------+
|age|   sex|   bmi|children|smoker|   region|    charges|
+---+------+------+--------+------+---------+-----------+
| 19|female|  27.9|       0|   yes|southwest|  16884.924|
| 18|  male| 33.77|       1|    no|southeast|  1725.5523|
| 28|  male|  33.0|       3|    no|southeast|   4449.462|
| 33|  male|22.705|       0|    no|northwest|21984.47061|
| 32|  male| 28.88|       0|    no|northwest|  3866.8552|
| 31|female| 25.74|       0|    no|southeast|  3756.6216|
| 46|female| 33.44|       1|    no|southeast|  8240.5896|
| 37|female| 27.74|       3|    no|northwest|  7281.5056|
| 37|  